# 03_modeling
Fit mixed-effects models and Cox proportional hazards models.


In [ ]:
import os, sys
repo_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
src_path = os.path.join(repo_root, "src")
if src_path not in sys.path:
    sys.path.insert(0, src_path)

import pandas as pd
from stats_utils import fit_mixedlm, fit_coxph
from data_utils import load_csv


In [ ]:
PROCESSED_DIR = os.path.join(repo_root, "data", "processed")
df_clin = pd.read_csv(os.path.join(PROCESSED_DIR, "clinical_cleaned.csv"))
df_surv = pd.read_csv(os.path.join(PROCESSED_DIR, "survival_cleaned.csv"))


In [ ]:
# Mixed effects: example formula 'weight ~ day * group'
# Ensure columns exist, otherwise adapt formula
if set(["weight","day","group","animalid"]).issubset(df_clin.columns):
    res = fit_mixedlm(df_clin, formula="weight ~ day * group", group_col="animalid", re_formula="~day")
    print(res.summary())
else:
    print("Columns required for mixed model not present. Columns:", df_clin.columns.tolist())


In [ ]:
# Cox PH: need duration & event + possible covariates (e.g., group)
if set(["time","event"]).issubset(df_surv.columns):
    # create dummy variables for categorical covariates if needed
    df_for_cox = df_surv.copy()
    if "group" in df_for_cox.columns:
        df_for_cox = pd.get_dummies(df_for_cox, columns=["group"], drop_first=True)
    covars = [c for c in df_for_cox.columns if c not in ["time","event","animalid"]]
    print("Fitting Cox with covariates:", covars)
    cph = fit_coxph(df_for_cox, duration_col="time", event_col="event", covariates=covars)
    print(cph.summary)
else:
    print("Survival columns missing; columns:", df_surv.columns.tolist())


## Next steps / saving results
- Save model summaries to text files or export coefficients.
- Create a results table with hazard ratios and CI for the manuscript.
